In [8]:
import numpy as np
import pandas as pd
import pymysql
import datetime as dt

In [2]:
# 开始链接
config = {
    'host':'rm-bp182zi1u38e0s8o8wo.mysql.rds.aliyuncs.com',
    'port':3306,
    'user':'meichai',
    'passwd':'VAKlVyh7F1eaDlKdUbLW',
    'db':'sale',
}
conn=pymysql.connect(**config)


In [3]:
# 显示表
cursor = conn.cursor()
cursor.execute('show tables')
data = cursor.fetchall()
data

(('sale_temp',),)

In [4]:
# 查询
cursor = conn.cursor()
sql=r'select * from sale_temp limit 1'
cursor.execute(sql)
columnDes = cursor.description
columnNames = [columnDes[i][0] for i in range(len(columnDes))]
df=pd.DataFrame()

i=1
limit=100000

while True:
    sql = "select * from sale_temp limit {}, {}".format(i, limit)
    cursor.execute(sql)
    rows = cursor.fetchall()

    if not len(rows):
        break
    
    df_temp=pd.DataFrame(list(rows))
    df=pd.concat([df,df_temp])
    

    i += limit
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1490203 entries, 0 to 90202
Data columns (total 12 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   0       1490203 non-null  int64         
 1   1       1490203 non-null  object        
 2   2       1490203 non-null  object        
 3   3       1490203 non-null  int64         
 4   4       1490203 non-null  object        
 5   5       1490203 non-null  int64         
 6   6       1490203 non-null  object        
 7   7       1490203 non-null  int64         
 8   8       1490203 non-null  object        
 9   9       1490203 non-null  object        
 10  10      1490203 non-null  datetime64[ns]
 11  11      1490203 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 113.7+ MB


In [5]:
df.columns=columnNames
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1490203 entries, 0 to 90202
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   id               1490203 non-null  int64         
 1   idcard           1490203 non-null  object        
 2   companyfullname  1490203 non-null  object        
 3   companysecid     1490203 non-null  int64         
 4   companysecname   1490203 non-null  object        
 5   channelid        1490203 non-null  int64         
 6   channelname      1490203 non-null  object        
 7   projectid        1490203 non-null  int64         
 8   projectname      1490203 non-null  object        
 9   payroll          1490203 non-null  object        
 10  paydate          1490203 non-null  datetime64[ns]
 11  risklevel        1490203 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 113.7+ MB


In [6]:
cursor.description

(('id', 3, None, 11, 11, 0, False),
 ('idcard', 253, None, 200, 200, 0, True),
 ('companyfullname', 253, None, 400, 400, 0, True),
 ('companysecid', 8, None, 20, 20, 0, True),
 ('companysecname', 253, None, 400, 400, 0, True),
 ('channelid', 8, None, 20, 20, 0, True),
 ('channelname', 253, None, 400, 400, 0, True),
 ('projectid', 8, None, 20, 20, 0, True),
 ('projectname', 253, None, 400, 400, 0, True),
 ('payroll', 246, None, 12, 12, 2, True),
 ('paydate', 12, None, 19, 19, 0, True),
 ('risklevel', 1, None, 4, 4, 0, True))

In [10]:
df_sep=df[df['paydate']>=dt.datetime(2020,9,1)]
df_sep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345729 entries, 30919 to 90202
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   id               345729 non-null  int64         
 1   idcard           345729 non-null  object        
 2   companyfullname  345729 non-null  object        
 3   companysecid     345729 non-null  int64         
 4   companysecname   345729 non-null  object        
 5   channelid        345729 non-null  int64         
 6   channelname      345729 non-null  object        
 7   projectid        345729 non-null  int64         
 8   projectname      345729 non-null  object        
 9   payroll          345729 non-null  object        
 10  paydate          345729 non-null  datetime64[ns]
 11  risklevel        345729 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 26.4+ MB


In [11]:
df_sep.iloc[1]

id                             6666577
idcard              510113199703315016
companyfullname         宁波快来人力资源管理有限公司
companysecid          1237922331391232
companysecname            四川新网银行股份有限公司
channelid             1186032750630144
channelname            抚州市昆硕网络科技服务有限公司
projectid             1245188655478016
projectname               四川新网银行股份有限公司
payroll                        1920.00
paydate            2020-09-16 14:22:05
risklevel                            4
Name: 30920, dtype: object

In [ ]:
for index,row in df.iterrows():
    if (df.idcard.value_counts()[row['idcard']]>1 and df[df['idcard']==row['idcard']].channelname.value_counts().shape[0]>1):
        df.loc[index,'flag']=1

In [33]:
# 关闭链接
cursor.close()
conn.close()

In [6]:
df.to_csv('history_data.csv',index=False)